<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Test_Notebook_Vilde_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Test Notebook Vilde: RCNN**



Trying to reproduce RCNN model example.

# **Setup**

In [0]:
!pip install git+git://github.com/gumpy-bci/gumpy.git

  Cloning git://github.com/gumpy-bci/gumpy.git to /tmp/pip-req-build-bb_idehn
  Running command git clone -q git://github.com/gumpy-bci/gumpy.git /tmp/pip-req-build-bb_idehn
  Created wheel for gumpy: filename=gumpy-0.5.0-cp36-none-any.whl size=32999 sha256=67be32d6795563dde6644f32fad840ef6e89b4b981ba40cfa3150d2ab2c1ceff
  Stored in directory: /tmp/pip-ephem-wheel-cache-p7t_v3py/wheels/cf/ee/4d/dfa04a0d04ffb1ca36f436f3fef4817e9e23bfec03dd772913
Successfully built gumpy


In [0]:
!git clone https://github.com/NathanielRose/eeg-294.git

Cloning into 'eeg-294'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 153 (delta 51), reused 84 (delta 11), pack-reused 0
Receiving objects: 100% (153/153), 1.70 MiB | 5.80 MiB/s, done.
Resolving deltas: 100% (51/51), done.


#### **Importing packages**


Importing module:

In [0]:
%reset
%matplotlib inline

from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys

import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Importing gumpy modules:

In [0]:
sys.path.append('gumpy/gumpy/')
import gumpy

sys.path.append('eeg-294/gumpy-deeplearning')
import models

sys.path.append('eeg-294/gumpy-deeplearning/models/')
import utils

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


#### **Setup parameters for the model and data**

In [0]:
DEBUG = True
######
## the RCNN FLAG because It does not use spectrograms but rather it uses directly the raw signals.
# so the data have something different compared to the spectrogram so you have to activte the flag.
######
RCNN_FLAG = True
###########

CLASS_COUNT = 2
DROPOUT = 0.2   # dropout rate in float

# parameters for filtering data
FS = 250
LOWCUT = 2
HIGHCUT = 60
ANTI_DRIFT = 0.5
CUTOFF = 50.0 # freq to be removed from signal (Hz) for notch filter
Q = 30.0  # quality factor for notch filter
W0 = CUTOFF/(FS/2)
AXIS = 0

#set random seed
SEED = 42
KFOLD = 5

#### **Load raw data**

In [0]:
%%bash
%%capture 

export fileid=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
export filename=grazdata.zip


wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

bash: line 1: fg: no job control
--2019-11-21 19:34:16--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
Resolving docs.google.com (docs.google.com)... 173.194.217.100, 173.194.217.139, 173.194.217.138, ...
Connecting to docs.google.com (docs.google.com)|173.194.217.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

     0K ...                                                    16.5M=0s

2019-11-21 19:34:16 (16.5 MB/s) - written to stdout [3248]

--2019-11-21 19:34:16--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb&confirm=ObM9
Resolving docs.google.com (docs.google.com)... 173.194.214.102, 173.194.214.101, 173.194.214.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.214.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-94-docs.googleusercontent.com/docs/securesc/ha0ro937gc

In [0]:
!unzip grazdata.zip

Archive:  grazdata.zip
mapname:  conversion of  failed
 extracting: B03T.mat                
 extracting: B05E.mat                
 extracting: B04T.mat                
 extracting: B01E.mat                
 extracting: B04E.mat                
 extracting: B06T.mat                
 extracting: B08T.mat                
 extracting: B07T.mat                
 extracting: B02T.mat                
 extracting: B03E.mat                
 extracting: B09T.mat                
 extracting: B05T.mat                
 extracting: B09E.mat                
 extracting: B01T.mat                
 extracting: B06E.mat                
 extracting: B07E.mat                
 extracting: B08E.mat                
 extracting: B02E.mat                


In [0]:
#load data
subject = 'B01'

grazb_data = gumpy.data.GrazB('', subject) # Initializing the data-structure
# grazb_data.load()


In [0]:
# now that the dataset is setup, we can load the data. This will be handled from within the utils function,
# which will first load the data and subsequently filter it using a notch and a bandpass filter.
# the utility function will then return the training data.
x_train, y_train = utils.load_preprocess_data(grazb_data, True, LOWCUT, HIGHCUT, W0, Q, ANTI_DRIFT, CLASS_COUNT, CUTOFF, AXIS, FS)

Band-pass filtering the data in frequency range from 2.0 Hz to 60.0 Hz... 
Data loaded and processed successfully!


#### **Augmented data**

In [0]:
x_augmented, y_augmented = gumpy.signal.sliding_window(data = x_train[:,:,:],
                                                          labels = y_train[:,:],
                                                          window_sz = 4 * FS,
                                                          n_hop = FS // 10,
                                                          n_start = FS * 1)
x_subject = x_augmented
y_subject = y_augmented
x_subject = np.rollaxis(x_subject, 2, 1)

#### **Run the model**

In [0]:
from sklearn.model_selection import StratifiedKFold
from models import RCNN

In [0]:
#from .model import KerasModel
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import BatchNormalization, Dropout, Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import merge, Conv2D, MaxPooling2D, Input
from keras.layers.advanced_activations import PReLU
from keras.layers import Activation
from keras.models import Model


In [0]:
# defining the K-fold cross validation 
kfold = StratifiedKFold(n_splits=KFOLD, shuffle=True, random_state=SEED)
cvscores = []
ii = 1
for train, test in kfold.split(x_subject, y_subject[:, 0]):
    print('Run ' + str(ii) + '...')


    model = RCNN(model_name_str)

    # create callbacks
    model_name_str = 'GRAZ_CNN_STFT_3layer_' + '_run_' + str(ii)
    #callbacks_list = model.get_callbacks(model_name_str)
    #print(x_subject.shape)
    #print(train)
    # initialize and create the model
    
    
    #model = RCNN(model_name_str)
    
    
    model.create_model(x_subject.shape[1:], print_summary=False, class_count = CLASS_COUNT)

    # fit model. If you specify monitor=True, then the model will create callbacks
    # and write its state to a HDF5 file

    if (RCNN_FLAG == True):
        x_subject = np.rollaxis(x_subject, 2, 1)
        x_subject = x_subject[:, np.newaxis, :, :]
        #print(x_subject.shape)
    model.fit(x_subject[train], y_subject[train], monitor=True,
              epochs=100,
              batch_size=256,
              verbose=0,
              validation_split=0.1,callbacks = callbacks_list)

    # evaluate the model
    print('Evaluating model on test set...')
    scores = model.evaluate(x_subject[test], y_subject[test], verbose=0)
    print("Result on test set: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    ii += 1

# print some evaluation statistics and write results to file
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
cv_all_subjects = np.asarray(cvscores)
print('Saving CV values to file....')
np.savetxt('GRAZ_CV_' + 'CNN_STFT_3layer_' + str(DROPOUT) + 'do' + '.csv',
           cv_all_subjects, delimiter=',', fmt='%2.4f')
print('CV values successfully saved!\n')

Run 1...


NameError: ignored